![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

## Oscilloscope 7

#### August 31, 2022

Based on Oscilloscope 5 and 6.

The goal: Remove all the HTML code, move the JS into a Jupyter cell, and run it without any external files. This should be easy. 

We also got this to run in Chrome, Safari and Firefox by using the method navigator.mediaDevices.getUserMedia, rather than the older and depricated version navigator.getUserMedia. This might cause some incompatibility with older browsers, though. 


In [5]:
import plotly.graph_objects as go
from IPython.display import Javascript

In [6]:
from IPython.display import display, HTML

In [7]:
myDataArray = [0,.1,.2,.3]

fig = go.Figure(data=go.Scatter( y=myDataArray, mode='lines'))
fig.update_layout(yaxis_range=[-.3,.3])
f = go.FigureWidget(fig)
f

FigureWidget({
    'data': [{'mode': 'lines', 'type': 'scatter', 'uid': '2a7e02ac-76d9-44f2-a948-699178fb39b6'…

In [8]:
Javascript("""

// We use the WebAudio API in Javascript in order to access the user's microphone. It asks for permission first. 

// Some variable to keep track of the audio context, source, and stream

var audioCtx = new (window.AudioContext || window.webkitAudioContext)();
var source;
var stream;

//set up the analyser node that grabs the sound samples

var analyser = audioCtx.createAnalyser();
    analyser.minDecibels = -90;
    analyser.maxDecibels = -10;
    analyser.smoothingTimeConstant = 0.85;

// a variable to keep track of animation frames

var drawVisual;

// start up the microphone (media) and then connect it to the visualizer

if (navigator.mediaDevices.getUserMedia) {
    console.log("mediaDevices.getUserMedia supported.");
    async function getMedia(constraints) {
        try {
            stream = await navigator.mediaDevices.getUserMedia(constraints);
            source = audioCtx.createMediaStreamSource(stream);
            source.connect(analyser);
            visualize();
        } 
        catch (err) {
            console.log("The following gUM error occured: " + err);
        }
    }
    getMedia({audio: true,})
} else {
          console.log("mediaDevices.getUserMedia not supported on your browser!");
};

// This visualize function catches the sound samples then passes it to the Plotly device in Python
// We also check one global variable "killDraw" to see when we should stop the drawing. 
function visualize() {
    analyser.fftSize = 1024;
    var dataArray = new Float32Array(analyser.fftSize);
    var fps = 4;
    window.killDraw = false;
    function draw(timestamp){
        if (window.killDraw) {return;}
        setTimeout(function(){ //throttle requestAnimationFrame to 4 fps
            analyser.getFloatTimeDomainData(dataArray);
            IPython.notebook.kernel.execute(
                "f.data[0]['y'] = "+ dataArray + ";"
            ); 
            drawVisual = requestAnimationFrame(draw)
        }, 1000/fps)
    }
    draw();
}

// finally, we save the "visualize" function in a variable we can call from a Jupyter cell later. 
window.myVis = visualize;

""")

<IPython.core.display.Javascript object>

## Starting and stopping

We include two cells to start and stop the drawing. Otherwise, the drawing is running all the time and never stops working. 

In [4]:
## Run this cell to stop the drawing
Javascript("window.killDraw=true;")

<IPython.core.display.Javascript object>

In [5]:
## Run this cell to restart the drawing
Javascript("window.killDraw=true; window.myVis();")

<IPython.core.display.Javascript object>

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)

In [13]:
%%js
window.killDraw = true;
console.log(window.killDraw)

<IPython.core.display.Javascript object>